In [1]:
%run _standard_imports.ipynb
%run _plotting_setup.ipynb

python 3.5.2 |Continuum Analytics, Inc.| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
numpy 1.11.2
scipy 0.18.1
pandas 0.19.0
numexpr 2.6.1
pysam 0.8.4
petl 1.1.0
petlx 1.0.3
vcf 0.6.8
vcfnp 2.2.0
h5py 2.6.0
tables 3.3.0


In [2]:
output_dir = '/lustre/scratch109/malaria/rp7/data/methods-dev/builds/Pf6.0/20161212_sample_level_summaries'
!mkdir -p {output_dir}/sample_summaries/Pf60
!mkdir -p {output_dir}/sample_summaries/Pv30
!mkdir -p {output_dir}/scripts
!mkdir -p {output_dir}/log

GENOME_FN = collections.OrderedDict()
genome_fn = collections.OrderedDict()
genome = collections.OrderedDict()
GENOME_FN['Pf60'] = "/lustre/scratch116/malaria/pfalciparum/resources/Pfalciparum.genome.fasta"
GENOME_FN['Pv30'] = "/lustre/scratch109/malaria/pvivax/resources/gatk/PvivaxP01.genome.fasta"
genome_fn['Pf60'] = "%s/Pfalciparum.genome.fasta" % output_dir
genome_fn['Pv30'] = "%s/PvivaxP01.genome.fasta" % output_dir

run_create_sample_summary_job_fn = "%s/scripts/run_create_sample_summary_job.sh" % output_dir
submit_create_sample_summary_jobs_fn = "%s/scripts/submit_create_sample_summary_jobs.sh" % output_dir


# sites_annotation_pf60_fn = '/lustre/scratch109/malaria/rp7/data/methods-dev/builds/Pf6.0/20161125_Pf60_final_vcfs/vcf/SNP_INDEL_WG.combined.filtered.annotation.vcf.gz'
hdf_fn = collections.OrderedDict()
hdf_fn['Pf60'] = '/lustre/scratch111/malaria/rp7/data/methods-dev/builds/Pf6.0/20161128_HDF5_build/hdf5/Pf_60_npy_PID_a12.h5'
hdf_fn['Pv30'] = '/lustre/scratch111/malaria/rp7/data/methods-dev/builds/Pf6.0/20161201_Pv_30_HDF5_build/hdf5/Pv_30.h5'


In [3]:
for release in GENOME_FN:
    !cp {GENOME_FN[release]} {genome_fn[release]}
    genome[release] = pyfasta.Fasta(genome_fn[release])
    print(sorted(genome[release].keys())[0])

Pf3D7_01_v3
PvP01_01_v1


In [4]:
hdf = collections.OrderedDict()
for release in hdf_fn:
    hdf[release] = h5py.File(hdf_fn[release], 'r')
    print(release, len(hdf[release]['samples']))
    

Pf60 7182
Pv30 1001


In [33]:
%%time
import pickle
import allel
calldata_subset = collections.OrderedDict()
for release in hdf_fn:
    calldata_subset[release] = collections.OrderedDict()
    for variable in ['genotype', 'GQ', 'DP', 'PGT']:
        calldata_subset[release][variable] = collections.OrderedDict()
        calldata = allel.GenotypeChunkedArray(hdf[release]['calldata'][variable])
        
        calldata_subset_fn = "%s/calldata_subset_%s_%s_first.p" % (output_dir, release, variable)
        if os.path.exists(calldata_subset_fn):
            print('loading', release, variable, 'first')
            calldata_subset[release][variable]['first'] = pickle.load(open(calldata_subset_fn, 'rb'))
        else:
            print('creating', release, variable, 'first')
            calldata_subset[release][variable]['first'] = calldata.subset(
                (hdf[release]['variants']['CHROM'][:] == sorted(genome[release].keys())[0].encode('ascii'))
            )
            
        calldata_subset_fn = "%s/calldata_subset_%s_%s_first_pass.p" % (output_dir, release, variable)
        if os.path.exists(calldata_subset_fn):
            print('loading', release, variable, 'first_pass')
            calldata_subset[release][variable]['first_pass'] = pickle.load(open(calldata_subset_fn, 'rb'))
        else:
            print('creating', release, variable, 'first_pass')
            calldata_subset[release][variable]['first_pass'] = calldata.subset(
                (hdf[release]['variants']['FILTER_PASS'][:]) &
                (hdf[release]['variants']['CHROM'][:] == sorted(genome[release].keys())[0].encode('ascii'))
            )
            
#         calldata_subset_fn = "%s/calldata_subset_%s_%s_pass.p" % (output_dir, release, variable)
#         if os.path.exists(calldata_subset_fn):
#             print('loading', release, variable, 'pass')
#             calldata_subset[release][variable]['pass'] = pickle.load(open(calldata_subset_fn, 'rb'))
#         else:
#             print('creating', release, variable, 'pass')
#             calldata_subset[release][variable]['pass'] = calldata.subset(hdf[release]['variants']['FILTER_PASS'][:])
            
#         calldata_subset_fn = "%s/calldata_subset_%s_%s_all.p" % (output_dir, release, variable)
#         if os.path.exists(calldata_subset_fn):
#             print('loading', release, variable, 'all')
#             calldata_subset[release][variable]['all'] = pickle.load(open(calldata_subset_fn, 'rb'))
#         else:
#             print('creating', release, variable, 'all')
#             calldata_subset[release][variable]['all'] = calldata.subset()
            
#             print(release, 'pass')
#             calldata_subset[release][variable]['pass'] = calldata.subset(hdf[release]['variants']['FILTER_PASS'][:])
#             print(release, 'all')
#             calldata_subset[release][variable]['all'] = calldata.subset()
        pickle.dump(genotypes_subset, open(genotypes_subset_fn, "wb"))


creating Pf60 genotype first
creating Pf60 genotype first_pass


TypeError: bad number of dimensions: expected 3; found 2

In [29]:
# %%time
# import pickle
# genotypes_subset_fn = "%s/genotypes_subset.p" % output_dir
# if os.path.exists(genotypes_subset_fn):
#     genotypes_subset = pickle.load(open(genotypes_subset_fn, "rb"))
# else:
#     genotypes = collections.OrderedDict()
#     genotypes_subset = collections.OrderedDict()
#     import allel
#     for release in hdf_fn:
#     # for release in ['Pv30']:
#         genotypes[release] = allel.GenotypeChunkedArray(hdf[release]['calldata']['genotype'])
#         genotypes_subset[release] = collections.OrderedDict()
#         print(release, 'first')
#         genotypes_subset[release]['first'] = genotypes[release].subset(
#             (hdf[release]['variants']['FILTER_PASS'][:]) &
#             (hdf[release]['variants']['CHROM'][:] == sorted(genome[release].keys())[0].encode('ascii'))
#         )
#         print(release, 'pass')
#         genotypes_subset[release]['pass'] = genotypes[release].subset(hdf[release]['variants']['FILTER_PASS'][:])
#         print(release, 'all')
#         genotypes_subset[release]['all'] = genotypes[release].subset()
#     pickle.dump(genotypes_subset, open(genotypes_subset_fn, "wb"))


In [5]:
%%time
import pickle
GQ_subset_fn = "%s/GQ_subset.p" % output_dir
if os.path.exists(GQ_subset_fn):
    genotypes_subset = pickle.load(GQ_subset_fn)
else:
    GQ = collections.OrderedDict()
    GQ_subset = collections.OrderedDict()
    for release in hdf_fn:
        GQ[release] = allel.GenotypeChunkedArray(hdf[release]['calldata']['GQ'])
        GQ_subset[release] = collections.OrderedDict()
        print(release, 'first')
        GQ_subset[release]['first'] = GQ[release].subset(
            (hdf[release]['variants']['FILTER_PASS'][:]) &
            (hdf[release]['variants']['CHROM'][:] == sorted(genome[release].keys())[0].encode('ascii'))
        )
        print(release, 'pass')
        GQ_subset[release]['pass'] = GQ[release].subset(hdf[release]['variants']['FILTER_PASS'][:])
        print(release, 'all')
        GQ_subset[release]['all'] = GQ[release].subset()
    pickle.dump(GQ_subset, open(GQ_subset_fn, "wb"))


Pf60 first
Pf60 pass
Pf60 all
Pv30 first
Pv30 pass
Pv30 all


TypeError: file must have a 'write' attribute

In [6]:
pickle.dump(genotypes_subset, open(genotypes_subset_fn, "wb"))

In [33]:
temp = (
    (hdf[release]['variants']['FILTER_PASS'][:]) &
    (hdf[release]['variants']['CHROM'][:] == sorted(genome[release].keys())[0].encode('ascii'))
)
pd.value_counts(temp)

False    4037882
True       46537
dtype: int64

In [30]:
sorted(genome[release].keys())[0].encode('ascii')

b'PvP01_01_v1'

In [28]:
pd.value_counts(hdf[release]['variants']['CHROM'][:])

b'Transfer.PvP'    1593135
b'PvP01_12_v1'      304616
b'PvP01_09_v1'      264054
b'PvP01_14_v1'      250792
b'PvP01_10_v1'      199272
b'PvP01_07_v1'      188371
b'PvP01_11_v1'      183502
b'PvP01_13_v1'      183133
b'PvP01_08_v1'      168624
b'PvP01_05_v1'      161750
b'PvP01_02_v1'      141169
b'PvP01_04_v1'      137416
b'PvP01_01_v1'      125338
b'PvP01_06_v1'       92927
b'PvP01_03_v1'       86345
b'PvP01_API_v1'       3268
b'PvP01_MIT_v1'        707
dtype: int64

In [14]:
hdf[release]['samples'][:]

array([b'Brazil01', b'Brazil02', b'Brazil03', ..., b'malagasy19',
       b'malagasy8', b'mauritania'], 
      dtype='|S22')

In [ ]:
genotypes_subset['Pf60']

In [ ]:
genotypes_subset['Pv30']

In [47]:
import allel
def create_sample_summary(hdf5_fn=hdf_fn['Pf60'], index=0, output_filestem="%s/sample_summaries/Pf60" % output_dir):
    results = collections.OrderedDict()
    
    hdf = h5py.File(hdf5_fn, 'r')
    samples = hdf['samples'][:]
    results['sample_id'] = samples[index].decode('ascii')
    output_fn = "%s/%s.txt" % (output_filestem, results['sample_id'])
    fo = open(output_fn, 'w')
    print(0)
    is_pass = hdf['variants']['FILTER_PASS'][:]
    
#     genotypes = allel.GenotypeChunkedArray(hdf['calldata']['genotype'])
    print(1)
    genotypes = allel.GenotypeArray(hdf['calldata']['genotype'][:, [index], :])
    print(2)
    genotypes_pass = genotypes[is_pass]
    
    print(3)
    svlen = hdf['variants']['svlen'][:]
    svlen1 = svlen[np.arange(svlen.shape[0]), genotypes[:, 0, 0] - 1]
    svlen1[np.in1d(genotypes[:, 0, 0], [-1, 0])] = 0 # Ref and missing are considered non-SV
    svlen2 = svlen[np.arange(svlen.shape[0]), genotypes[:, 0, 1] - 1]
    svlen2[np.in1d(genotypes[:, 0, 1], [-1, 0])] = 0 # Ref and missing are considered non-SV
    
    print(4)
    is_snp = (hdf['variants']['VARIANT_TYPE'][:][is_pass] == b'SNP')
    is_bi = (hdf['variants']['MULTIALLELIC'][:][is_pass] == b'BI')
    is_sd = (hdf['variants']['MULTIALLELIC'][:][is_pass] == b'SD')
    is_mu = (hdf['variants']['MULTIALLELIC'][:][is_pass] == b'MU')
    is_ins = ((svlen1 > 0) | (svlen2 > 0))[is_pass]
    is_del = ((svlen1 < 0) | (svlen2 < 0))[is_pass]
    is_ins_del_het = (((svlen1 > 0) & (svlen2 < 0)) | ((svlen1 < 0) & (svlen2 > 0)))[is_pass] # These are hets where one allele is insertion and the other allele is deletion (these are probably rare)
    
    print(5)
    results['num_variants']      = genotypes.shape[0]
    results['num_pass_variants'] = np.count_nonzero(is_pass)
    results['num_missing']       = genotypes.count_missing(axis=0)[0]
    results['num_pass_missing']  = genotypes_pass.count_missing(axis=0)[0]
    results['num_called']        = (results['num_variants'] - results['num_missing'])
    results['num_pass_called']   = (results['num_pass_variants'] - results['num_pass_missing'])
    print(6)
    results['num_het']           = genotypes.count_het(axis=0)[0]
    results['num_pass_het']      = genotypes_pass.count_het(axis=0)[0]
    results['num_hom_alt']       = genotypes.count_hom_alt(axis=0)[0]
    results['num_pass_hom_alt']  = genotypes_pass.count_hom_alt(axis=0)[0]
    print(7)
    results['num_snp_hom_ref']   = genotypes_pass.subset(is_snp).count_hom_ref(axis=0)[0]
    results['num_snp_het']       = genotypes_pass.subset(is_snp).count_het(axis=0)[0]
    results['num_snp_hom_alt']   = genotypes_pass.subset(is_snp).count_hom_alt(axis=0)[0]
    results['num_indel_hom_ref'] = genotypes_pass.subset(~is_snp).count_hom_ref(axis=0)[0]
    results['num_indel_het']     = genotypes_pass.subset(~is_snp).count_het(axis=0)[0]
    results['num_indel_hom_alt'] = genotypes_pass.subset(~is_snp).count_hom_alt(axis=0)[0]
    print(8)    
    results['num_ins_hom_ref']   = genotypes_pass.subset(is_ins).count_hom_ref(axis=0)[0]
    results['num_ins_het']       = genotypes_pass.subset(is_ins).count_het(axis=0)[0]
    results['num_ins']           = (results['num_ins_hom_ref'] + results['num_ins_het'])
    results['num_del_hom_ref']   = genotypes_pass.subset(is_del).count_hom_ref(axis=0)[0]
    results['num_del_het']       = genotypes_pass.subset(is_del).count_het(axis=0)[0]
    results['num_del']           = (results['num_del_hom_ref'] + results['num_del_het'])
    
    print(9)
    results['pc_pass']           = results['num_pass_called'] / results['num_called']
    results['pc_missing']        = results['num_missing'] / results['num_variants']
    results['pc_pass_missing']   = results['num_pass_missing'] / results['num_pass_variants']
    results['pc_het']            = results['num_het'] / results['num_called']
    results['pc_pass_het']       = results['num_pass_het'] / results['num_pass_called']
    results['pc_hom_alt']        = results['num_hom_alt'] / results['num_called']
    results['pc_pass_hom_alt']   = results['num_pass_hom_alt'] / results['num_pass_called']
    results['pc_snp']            = (results['num_snp_het'] + results['num_snp_hom_alt']) / (results['num_snp_het'] + results['num_snp_hom_alt'] + results['num_indel_het'] + results['num_indel_hom_alt'])
    results['pc_ins']            = (results['num_ins'] / (results['num_ins'] + results['num_del']))

    print(10)
    
    print('\t'.join([str(x) for x in list(results.keys())]), file=fo)
    print('\t'.join([str(x) for x in list(results.values())]), file=fo)
    fo.close()
    
    df_sample_summary = pd.DataFrame(
            {
                'Sample': pd.Series(results['sample_id']),
                'Variants called': pd.Series(results['num_called']),
                'Variants missing': pd.Series(results['num_missing']),
                'Proportion missing': pd.Series(results['pc_missing']),
                'Proportion pass missing': pd.Series(results['pc_pass_missing']),
                'Proportion heterozygous': pd.Series(results['pc_het']),
                'Proportion pass heterozygous': pd.Series(results['pc_pass_het']),
                'Proportion homozygous alternative': pd.Series(results['pc_hom_alt']),
                'Proportion pass homozygous alternative': pd.Series(results['pc_pass_hom_alt']),
                'Proportion variants SNPs': pd.Series(results['pc_snp']),
                'Proportion indels insertions': pd.Series(results['pc_ins']),
            }
        )  
    return(df_sample_summary, results)

In [49]:
hdf_fn['Pf60']

'/lustre/scratch111/malaria/rp7/data/methods-dev/builds/Pf6.0/20161128_HDF5_build/hdf5/Pf_60_npy_PID_a12.h5'

In [167]:
import allel
def create_sample_summary(index=0, hdf5_fn='/lustre/scratch111/malaria/rp7/data/methods-dev/builds/Pf6.0/20161128_HDF5_build/hdf5/Pf_60_npy_PID_a12.h5'):
    results = collections.OrderedDict()
    
    hdf = h5py.File(hdf5_fn, 'r')
    samples = hdf['samples'][:]
    results['sample_id'] = samples[index].decode('ascii')
#     output_fn = "%s/%s.txt" % (output_filestem, results['sample_id'])
#     fo = open(output_fn, 'w')
    is_pass = hdf['variants']['FILTER_PASS'][:]
    
    genotypes = allel.GenotypeArray(hdf['calldata']['genotype'][:, [index], :])
    genotypes_pass = genotypes[is_pass]
    
    svlen = hdf['variants']['svlen'][:]
    svlen1 = svlen[np.arange(svlen.shape[0]), genotypes[:, 0, 0] - 1]
    svlen1[np.in1d(genotypes[:, 0, 0], [-1, 0])] = 0 # Ref and missing are considered non-SV
    svlen2 = svlen[np.arange(svlen.shape[0]), genotypes[:, 0, 1] - 1]
    svlen2[np.in1d(genotypes[:, 0, 1], [-1, 0])] = 0 # Ref and missing are considered non-SV
    
    ac = hdf['variants']['AC'][:]
    ac1 = ac[np.arange(ac.shape[0]), genotypes[:, 0, 0] - 1]
    ac1[np.in1d(genotypes[:, 0, 0], [-1, 0])] = 0 # Ref and missing are considered non-singleton
    ac2 = ac[np.arange(ac.shape[0]), genotypes[:, 0, 1] - 1]
    ac2[np.in1d(genotypes[:, 0, 1], [-1, 0])] = 0 # Ref and missing are considered non-singleton
    
    is_snp = (hdf['variants']['VARIANT_TYPE'][:][is_pass] == b'SNP')
    is_bi = (hdf['variants']['MULTIALLELIC'][:][is_pass] == b'BI')
    is_sd = (hdf['variants']['MULTIALLELIC'][:][is_pass] == b'SD')
    is_mu = (hdf['variants']['MULTIALLELIC'][:][is_pass] == b'MU')
    is_ins = ((svlen1 > 0) | (svlen2 > 0))[is_pass]
    is_del = ((svlen1 < 0) | (svlen2 < 0))[is_pass]
    is_ins_del_het = (((svlen1 > 0) & (svlen2 < 0)) | ((svlen1 < 0) & (svlen2 > 0)))[is_pass] # These are hets where one allele is insertion and the other allele is deletion (these are probably rare)
    is_coding = (hdf['variants']['CDS'][:][is_pass])
    is_vqslod6 = (hdf['variants']['VQSLOD'][:][is_pass] >= 6.0)
    is_vhq_snp = (is_vqslod6 & is_snp & is_bi & is_coding)
    is_nonsynonymous = (hdf['variants']['SNPEFF_EFFECT'][:][is_pass] == b'NON_SYNONYMOUS_CODING')
    is_synonymous = (hdf['variants']['SNPEFF_EFFECT'][:][is_pass] == b'SYNONYMOUS_CODING')
    is_frameshift = (hdf['variants']['SNPEFF_EFFECT'][:][is_pass] == b'FRAME_SHIFT')
    is_inframe = np.in1d(hdf['variants']['SNPEFF_EFFECT'][:][is_pass], [b'CODON_INSERTION', b'CODON_DELETION', b'CODON_CHANGE_PLUS_CODON_DELETION', b'CODON_CHANGE_PLUS_CODON_INSERTION'])

    is_singleton = (
        ((ac1 == 1) & (genotypes[:, 0, 0] > 0)) |
        ((ac2 == 1) & (genotypes[:, 0, 1] > 0)) |
        ((ac1 == 2) & (genotypes[:, 0, 0] > 0) & (genotypes[:, 0, 1] > 0))
    )[is_pass]
    
    is_pass_nonref = (is_pass & ((genotypes[:, 0, 0] > 0) | (genotypes[:, 0, 1] > 0)))
    is_biallelic_snp_nonref = (is_snp & is_bi &((genotypes_pass[:, 0, 0] > 0) | (genotypes_pass[:, 0, 1] > 0)))
    is_biallelic_indel_nonref = (~is_snp & is_bi &((genotypes_pass[:, 0, 0] > 0) | (genotypes_pass[:, 0, 1] > 0)))
    
    GQ = hdf['calldata']['GQ'][:, [index]][is_pass]
    DP = hdf['calldata']['DP'][:, [index]][is_pass]
    PGT = hdf['calldata']['PGT'][:, [index]][is_pass]
    
    mutations = np.char.add(hdf['variants']['REF'][:][is_pass][is_biallelic_snp_nonref], hdf['variants']['ALT'][:, 0][is_pass][is_biallelic_snp_nonref])
    is_transition = np.in1d(mutations, [b'AG', b'GA', b'CT', b'TC'])
    is_transversion = np.in1d(mutations, [b'AC', b'AT', b'GC', b'GT', b'CA', b'CG', b'TA', b'TG'])
    is_AT_to_AT = np.in1d(mutations, [b'AT', b'TA'])
    is_CG_to_CG = np.in1d(mutations, [b'CG', b'GC'])
    is_AT_to_CG = np.in1d(mutations, [b'AC', b'AG', b'TC', b'TG'])
    is_CG_to_AT = np.in1d(mutations, [b'CA', b'GA', b'CT', b'GT'])

    results['num_variants']             = genotypes.shape[0]
    results['num_pass_variants']        = np.count_nonzero(is_pass)
    results['num_missing']              = genotypes.count_missing(axis=0)[0]
    results['num_pass_missing']         = genotypes_pass.count_missing(axis=0)[0]
    results['num_called']               = (results['num_variants'] - results['num_missing'])
    results['num_pass_called']          = (results['num_pass_variants'] - results['num_pass_missing'])

    results['num_het']                  = genotypes.count_het(axis=0)[0]
    results['num_pass_het']             = genotypes_pass.count_het(axis=0)[0]
    results['num_hom_alt']              = genotypes.count_hom_alt(axis=0)[0]
    results['num_pass_hom_alt']         = genotypes_pass.count_hom_alt(axis=0)[0]
#     results['num_pass_non_ref']         = (results['num_pass_het'] + results['num_pass_hom_alt'])
    results['num_pass_non_ref']         = np.count_nonzero(is_pass_nonref)
    
    results['num_biallelic_het']        = genotypes_pass.subset(is_bi).count_het(axis=0)[0]
    results['num_biallelic_hom_alt']    = genotypes_pass.subset(is_bi).count_hom_alt(axis=0)[0]
    results['num_spanning_del_het']     = genotypes_pass.subset(is_sd).count_het(axis=0)[0]
    results['num_spanning_del_hom_alt'] = genotypes_pass.subset(is_sd).count_hom_alt(axis=0)[0]
    results['num_multiallelic_het']     = genotypes_pass.subset(is_mu).count_het(axis=0)[0]
    results['num_multiallelic_hom_alt'] = genotypes_pass.subset(is_mu).count_hom_alt(axis=0)[0]
    
    results['num_snp_hom_ref']          = genotypes_pass.subset(is_snp).count_hom_ref(axis=0)[0]
    results['num_snp_het']              = genotypes_pass.subset(is_snp).count_het(axis=0)[0]
    results['num_snp_hom_alt']          = genotypes_pass.subset(is_snp).count_hom_alt(axis=0)[0]
    results['num_indel_hom_ref']        = genotypes_pass.subset(~is_snp).count_hom_ref(axis=0)[0]
    results['num_indel_het']            = genotypes_pass.subset(~is_snp).count_het(axis=0)[0]
    results['num_indel_hom_alt']        = genotypes_pass.subset(~is_snp).count_hom_alt(axis=0)[0]

    results['num_ins_het']              = genotypes_pass.subset(is_ins).count_het(axis=0)[0]
    results['num_ins_hom_alt']          = genotypes_pass.subset(is_ins).count_hom_alt(axis=0)[0]
    results['num_ins']                  = (results['num_ins_hom_alt'] + results['num_ins_het'])
    results['num_del_het']              = genotypes_pass.subset(is_del).count_het(axis=0)[0]
    results['num_del_hom_alt']          = genotypes_pass.subset(is_del).count_hom_alt(axis=0)[0]
    results['num_del']                  = (results['num_del_hom_alt'] + results['num_del_het'])
    
    results['num_coding_het']           = genotypes_pass.subset(is_coding).count_het(axis=0)[0]
    results['num_coding_hom_alt']       = genotypes_pass.subset(is_coding).count_hom_alt(axis=0)[0]
    results['num_coding']               = (results['num_coding_het'] + results['num_coding_hom_alt'])
    
    results['num_vhq_snp_hom_ref']      = genotypes_pass.subset(is_vhq_snp).count_hom_ref(axis=0)[0]
    results['num_vhq_snp_het']          = genotypes_pass.subset(is_vhq_snp).count_het(axis=0)[0]
    results['num_vhq_snp_hom_alt']      = genotypes_pass.subset(is_vhq_snp).count_hom_alt(axis=0)[0]
    
    results['num_singleton']            = np.count_nonzero(is_singleton)
    results['num_biallelic_singleton']  = np.count_nonzero(is_bi & is_singleton)
    results['num_vhq_snp_singleton']    = np.count_nonzero(is_vhq_snp & is_singleton)

    results['num_bi_nonsynonymous']     = np.count_nonzero(is_biallelic_snp_nonref & is_nonsynonymous)
    results['num_bi_synonymous']        = np.count_nonzero(is_biallelic_snp_nonref & is_synonymous)
    results['num_bi_frameshift']        = np.count_nonzero(is_biallelic_indel_nonref & is_frameshift)
    results['num_bi_inframe']           = np.count_nonzero(is_biallelic_indel_nonref & is_inframe)

    results['num_bi_transition']        = np.count_nonzero(is_transition)
    results['num_bi_transversion']      = np.count_nonzero(is_transversion)
    results['num_bi_AT_to_AT']          = np.count_nonzero(is_AT_to_AT)
    results['num_bi_CG_to_CG']          = np.count_nonzero(is_CG_to_CG)
    results['num_bi_AT_to_CG']          = np.count_nonzero(is_AT_to_CG)
    results['num_bi_CG_to_AT']          = np.count_nonzero(is_CG_to_AT)

    results['pc_pass']                  = results['num_pass_called'] / results['num_called']
    results['pc_missing']               = results['num_missing'] / results['num_variants']
    results['pc_pass_missing']          = results['num_pass_missing'] / results['num_pass_variants']
    results['pc_het']                   = results['num_het'] / results['num_called']
    results['pc_pass_het']              = results['num_pass_het'] / results['num_pass_called']
    results['pc_hom_alt']               = results['num_hom_alt'] / results['num_called']
    results['pc_pass_hom_alt']          = results['num_pass_hom_alt'] / results['num_pass_called']
    results['pc_snp']                   = (results['num_snp_het'] + results['num_snp_hom_alt']) / results['num_pass_non_ref']
#     results['pc_snp_v2']                = (results['num_snp_het'] + results['num_snp_hom_alt']) / (results['num_snp_het'] + results['num_snp_hom_alt'] + results['num_indel_het'] + results['num_indel_hom_alt'])
    results['pc_biallelic']             = (results['num_biallelic_het'] + results['num_biallelic_hom_alt']) / results['num_pass_non_ref']
    results['pc_spanning_del']          = (results['num_spanning_del_het'] + results['num_spanning_del_hom_alt']) / results['num_pass_non_ref']
    results['pc_mutliallelic']          = (results['num_multiallelic_het'] + results['num_multiallelic_hom_alt']) / results['num_pass_non_ref']
    results['pc_ins']                   = (results['num_ins'] / (results['num_ins'] + results['num_del']))
    results['pc_coding']                = results['num_coding'] / results['num_pass_non_ref']
#     results['pc_bi_nonsynonymous']      = results['num_bi_nonsynonymous'] / (results['num_biallelic_het'] + results['num_biallelic_hom_alt'])
    results['pc_bi_nonsynonymous']      = results['num_bi_nonsynonymous'] / (results['num_bi_nonsynonymous'] + results['num_bi_synonymous'])
    results['pc_bi_frameshift']         = results['num_bi_frameshift'] / (results['num_bi_frameshift'] + results['num_bi_inframe'])
    results['pc_bi_transition']         = results['num_bi_transition'] / (results['num_bi_transition'] + results['num_bi_transversion'])
    results['pc_bi_AT_to_AT']           = results['num_bi_AT_to_AT'] / (results['num_bi_AT_to_AT'] + results['num_bi_CG_to_CG'] + results['num_bi_AT_to_CG'] + results['num_bi_CG_to_AT'])
    results['pc_bi_CG_to_CG']           = results['num_bi_CG_to_CG'] / (results['num_bi_AT_to_AT'] + results['num_bi_CG_to_CG'] + results['num_bi_AT_to_CG'] + results['num_bi_CG_to_AT'])
    results['pc_bi_AT_to_CG']           = results['num_bi_AT_to_CG'] / (results['num_bi_AT_to_AT'] + results['num_bi_CG_to_CG'] + results['num_bi_AT_to_CG'] + results['num_bi_CG_to_AT'])
    results['pc_bi_CG_to_AT']           = results['num_bi_CG_to_AT'] / (results['num_bi_AT_to_AT'] + results['num_bi_CG_to_CG'] + results['num_bi_AT_to_CG'] + results['num_bi_CG_to_AT'])
    
    results['mean_GQ']                  = np.mean(GQ)
    results['mean_GQ_2']                = np.nanmean(GQ)
    results['mean_DP']                  = np.mean(DP)
    results['mean_DP_2']                = np.nanmean(DP)
    
    print('\t'.join([str(x) for x in list(results.keys())]))
    print('\t'.join([str(x) for x in list(results.values())]))

    return(results, PGT)

In [301]:
import allel
def create_sample_summary_2(index=0, hdf5_fn='/lustre/scratch111/malaria/rp7/data/methods-dev/builds/Pf6.0/20161128_HDF5_build/hdf5/Pf_60_npy_PID_a12.h5'):
    results = collections.OrderedDict()
    
    hdf = h5py.File(hdf5_fn, 'r')
    samples = hdf['samples'][:]
    results['sample_id'] = samples[index].decode('ascii')
#     output_fn = "%s/%s.txt" % (output_filestem, results['sample_id'])
#     fo = open(output_fn, 'w')
    is_pass = hdf['variants']['FILTER_PASS'][:]
    
    genotypes = allel.GenotypeArray(hdf['calldata']['genotype'][:, [index], :])
#     genotypes_pass = genotypes[is_pass]
    
    svlen = hdf['variants']['svlen'][:]
    svlen1 = svlen[np.arange(svlen.shape[0]), genotypes[:, 0, 0] - 1]
    svlen1[np.in1d(genotypes[:, 0, 0], [-1, 0])] = 0 # Ref and missing are considered non-SV
    svlen2 = svlen[np.arange(svlen.shape[0]), genotypes[:, 0, 1] - 1]
    svlen2[np.in1d(genotypes[:, 0, 1], [-1, 0])] = 0 # Ref and missing are considered non-SV
    indel_len = svlen1
    het_indels = (svlen1 != svlen2)
    indel_len[het_indels] = svlen1[het_indels] + svlen2[het_indels]
    
    is_indel = (indel_len != 0)
    is_inframe = ((indel_len != 0) & (indel_len%3 == 0))
    is_frameshift = ((indel_len != 0) & (indel_len%3 != 0))
    
    ac = hdf['variants']['AC'][:]
    ac1 = ac[np.arange(ac.shape[0]), genotypes[:, 0, 0] - 1]
    ac1[np.in1d(genotypes[:, 0, 0], [-1, 0])] = 0 # Ref and missing are considered non-singleton
    ac2 = ac[np.arange(ac.shape[0]), genotypes[:, 0, 1] - 1]
    ac2[np.in1d(genotypes[:, 0, 1], [-1, 0])] = 0 # Ref and missing are considered non-singleton
    
    is_snp = (hdf['variants']['VARIANT_TYPE'][:] == b'SNP')
    is_bi = (hdf['variants']['MULTIALLELIC'][:] == b'BI')
    is_sd = (hdf['variants']['MULTIALLELIC'][:] == b'SD')
    is_mu = (hdf['variants']['MULTIALLELIC'][:] == b'MU')
    is_ins = ((svlen1 > 0) | (svlen2 > 0))
    is_del = ((svlen1 < 0) | (svlen2 < 0))
    is_ins_del_het = (((svlen1 > 0) & (svlen2 < 0)) | ((svlen1 < 0) & (svlen2 > 0))) # These are hets where one allele is insertion and the other allele is deletion (these are probably rare)
    is_coding = (hdf['variants']['CDS'][:])
    is_vqslod6 = (hdf['variants']['VQSLOD'][:] >= 6.0)
    is_hq_snp = (is_pass & is_snp & is_bi & is_coding)
    is_vhq_snp = (is_pass & is_vqslod6 & is_snp & is_bi & is_coding)
    is_nonsynonymous = (hdf['variants']['SNPEFF_EFFECT'][:] == b'NON_SYNONYMOUS_CODING')
    is_synonymous = (hdf['variants']['SNPEFF_EFFECT'][:] == b'SYNONYMOUS_CODING')
    is_frameshift_snpeff = (hdf['variants']['SNPEFF_EFFECT'][:] == b'FRAME_SHIFT')
    is_inframe_snpeff = np.in1d(hdf['variants']['SNPEFF_EFFECT'][:], [b'CODON_INSERTION', b'CODON_DELETION', b'CODON_CHANGE_PLUS_CODON_DELETION', b'CODON_CHANGE_PLUS_CODON_INSERTION'])

    is_singleton = (
        ((ac1 == 1) & (genotypes[:, 0, 0] > 0)) |
        ((ac2 == 1) & (genotypes[:, 0, 1] > 0)) |
        ((ac1 == 2) & (genotypes[:, 0, 0] > 0) & (genotypes[:, 0, 1] > 0))
    )
    
    is_hom_ref = ((genotypes[:, 0, 0] == 0) & (genotypes[:, 0, 1] == 0))
    is_het = ((genotypes[:, 0, 0] != genotypes[:, 0, 1]))
    is_hom_alt = ((genotypes[:, 0, 0] > 0) & (genotypes[:, 0, 0] == genotypes[:, 0, 1]))
    is_non_ref = ((genotypes[:, 0, 0] > 0) | (genotypes[:, 0, 1] > 0))
    is_missing = ((genotypes[:, 0, 0] == -1))
    is_called = ((genotypes[:, 0, 0] >= 0))
    
    GQ = hdf['calldata']['GQ'][:, index]
    is_GQ_30 = (GQ >= 30)
    is_GQ_99 = (GQ >= 99)
    DP = hdf['calldata']['DP'][:, index]
    PGT = hdf['calldata']['PGT'][:, index]
    is_phased = np.in1d(PGT, [b'.', b''], invert=True)
    
    mutations = np.char.add(hdf['variants']['REF'][:][(is_pass & is_snp & is_bi & is_non_ref)], hdf['variants']['ALT'][:, 0][(is_pass & is_snp & is_bi & is_non_ref)])
    is_transition = np.in1d(mutations, [b'AG', b'GA', b'CT', b'TC'])
    is_transversion = np.in1d(mutations, [b'AC', b'AT', b'GC', b'GT', b'CA', b'CG', b'TA', b'TG'])
    is_AT_to_AT = np.in1d(mutations, [b'AT', b'TA'])
    is_CG_to_CG = np.in1d(mutations, [b'CG', b'GC'])
    is_AT_to_CG = np.in1d(mutations, [b'AC', b'AG', b'TC', b'TG'])
    is_CG_to_AT = np.in1d(mutations, [b'CA', b'GA', b'CT', b'GT'])

    results['num_variants']             = genotypes.shape[0]
    results['num_pass_variants']        = np.count_nonzero(is_pass)
    results['num_missing']              = np.count_nonzero(is_missing)
    results['num_pass_missing']         = np.count_nonzero(is_pass & is_missing)
    results['num_called']               = np.count_nonzero(~is_missing)
#     results['num_called']               = (results['num_variants'] - results['num_missing'])
    results['num_pass_called']          = np.count_nonzero(is_pass & is_called)
#     results['num_pass_called_2']        = np.count_nonzero(is_pass & ~is_missing)
#     results['num_pass_called']          = (results['num_pass_variants'] - results['num_pass_missing'])

    results['num_hom_ref']              = np.count_nonzero(is_hom_ref)
    results['num_het']                  = np.count_nonzero(is_het)
    results['num_pass_het']             = np.count_nonzero(is_pass & is_het)
    results['num_hom_alt']              = np.count_nonzero(is_hom_alt)
    results['num_pass_hom_alt']         = np.count_nonzero(is_pass & is_hom_alt)
#     results['num_pass_non_ref']         = (results['num_pass_het'] + results['num_pass_hom_alt'])
    results['num_pass_non_ref']         = np.count_nonzero(is_pass & is_non_ref)
#     results['num_variants_2']           = results['num_hom_ref'] + results['num_het'] + results['num_hom_alt'] + results['num_missing']
    
    results['num_biallelic_het']        = np.count_nonzero(is_pass & is_bi & is_het)
    results['num_biallelic_hom_alt']    = np.count_nonzero(is_pass & is_bi & is_hom_alt)
    results['num_spanning_del_het']     = np.count_nonzero(is_pass & is_sd & is_het)
    results['num_spanning_del_hom_alt'] = np.count_nonzero(is_pass & is_sd & is_hom_alt)
    results['num_multiallelic_het']     = np.count_nonzero(is_pass & is_mu & is_het)
    results['num_multiallelic_hom_alt'] = np.count_nonzero(is_pass & is_mu & is_hom_alt)
    
#     results['num_snp_hom_ref']          = np.count_nonzero(is_pass & is_snp & is_het)
    results['num_snp_het']              = np.count_nonzero(is_pass & is_snp & is_het)
    results['num_snp_hom_alt']          = np.count_nonzero(is_pass & is_snp & is_hom_alt)
    results['num_snp']                  = (results['num_snp_het'] + results['num_snp_hom_alt'])
#     results['num_indel_hom_ref']        = genotypes_pass.subset(~is_snp).count_hom_ref(axis=0)[0]
    results['num_indel_het']            = np.count_nonzero(is_pass & ~is_snp & is_het)
    results['num_indel_hom_alt']        = np.count_nonzero(is_pass & ~is_snp & is_het)
    results['num_indel']                  = (results['num_indel_het'] + results['num_indel_hom_alt'])

    results['num_ins_het']              = np.count_nonzero(is_pass & is_ins & is_het)
    results['num_ins_hom_alt']          = np.count_nonzero(is_pass & is_ins & is_hom_alt)
    results['num_ins']                  = (results['num_ins_hom_alt'] + results['num_ins_het'])
    results['num_del_het']              = np.count_nonzero(is_pass & is_del & is_het)
    results['num_del_hom_alt']          = np.count_nonzero(is_pass & is_del & is_hom_alt)
    results['num_del']                  = (results['num_del_hom_alt'] + results['num_del_het'])
    
    results['num_coding_het']           = np.count_nonzero(is_pass & is_coding & is_het)
    results['num_coding_hom_alt']       = np.count_nonzero(is_pass & is_coding & is_hom_alt)
    results['num_coding']               = (results['num_coding_het'] + results['num_coding_hom_alt'])
    
    results['num_hq_snp_called']        = np.count_nonzero(is_hq_snp & ~is_missing)
    results['num_hq_snp_hom_ref']       = np.count_nonzero(is_hq_snp & is_hom_ref)
    results['num_hq_snp_het']           = np.count_nonzero(is_hq_snp & is_het)
    results['num_hq_snp_hom_alt']       = np.count_nonzero(is_hq_snp & is_hom_alt)
    results['num_vhq_snp_called']       = np.count_nonzero(is_vhq_snp & ~is_missing)
    results['num_vhq_snp_hom_ref']      = np.count_nonzero(is_vhq_snp & is_hom_ref)
    results['num_vhq_snp_het']          = np.count_nonzero(is_vhq_snp & is_het)
    results['num_vhq_snp_hom_alt']      = np.count_nonzero(is_vhq_snp & is_hom_alt)
    
    results['num_singleton']            = np.count_nonzero(is_pass & is_singleton)
    results['num_biallelic_singleton']  = np.count_nonzero(is_pass & is_bi & is_singleton)
    results['num_hq_snp_singleton']     = np.count_nonzero(is_hq_snp & is_singleton)
    results['num_vhq_snp_singleton']    = np.count_nonzero(is_vhq_snp & is_singleton)

    results['num_bi_nonsynonymous']     = np.count_nonzero(is_pass & is_bi & is_snp & is_non_ref & is_nonsynonymous)
    results['num_bi_synonymous']        = np.count_nonzero(is_pass & is_bi & is_snp & is_non_ref & is_synonymous)
#     results['num_frameshift']           = np.count_nonzero(is_pass & is_indel & is_non_ref & is_coding & is_frameshift)
#     results['num_inframe']              = np.count_nonzero(is_pass & is_indel & is_non_ref & is_coding & is_inframe)
    results['num_frameshift']           = np.count_nonzero(is_pass & is_indel & is_coding & is_frameshift)
    results['num_inframe']              = np.count_nonzero(is_pass & is_indel & is_coding & is_inframe)
    results['num_bi_frameshift']        = np.count_nonzero(is_pass & is_bi & is_indel & is_coding & is_non_ref & is_frameshift)
    results['num_bi_inframe']           = np.count_nonzero(is_pass & is_bi & is_indel & is_coding & is_non_ref & is_inframe)
    results['num_hq_frameshift']        = np.count_nonzero(is_pass & is_vqslod6 & is_bi & is_indel & is_coding & is_non_ref & is_frameshift)
    results['num_hq_inframe']           = np.count_nonzero(is_pass & is_vqslod6 & is_bi & is_indel & is_coding & is_non_ref & is_inframe)
    results['num_bi_frameshift_snpeff'] = np.count_nonzero(is_pass & is_bi & ~is_snp & is_non_ref & is_frameshift_snpeff)
    results['num_bi_inframe_snpeff']    = np.count_nonzero(is_pass & is_bi & ~is_snp & is_non_ref & is_inframe_snpeff)

    results['num_bi_transition']        = np.count_nonzero(is_transition)
    results['num_bi_transversion']      = np.count_nonzero(is_transversion)
    results['num_bi_AT_to_AT']          = np.count_nonzero(is_AT_to_AT)
    results['num_bi_CG_to_CG']          = np.count_nonzero(is_CG_to_CG)
    results['num_bi_AT_to_CG']          = np.count_nonzero(is_AT_to_CG)
    results['num_bi_CG_to_AT']          = np.count_nonzero(is_CG_to_AT)

    results['num_phased']               = np.count_nonzero(is_pass & is_phased)
    results['num_phased_non_ref']       = np.count_nonzero(is_pass & is_phased & is_non_ref)
    results['num_phased_hom_ref']       = np.count_nonzero(is_pass & is_phased & is_hom_ref)
    results['num_phased_missing']       = np.count_nonzero(is_pass & is_phased & is_missing)
    
    results['num_GQ_30']                = np.count_nonzero(is_pass & is_called & is_GQ_30)
    results['num_het_GQ_30']            = np.count_nonzero(is_pass & is_het & is_GQ_30)
    results['num_hom_alt_GQ_30']        = np.count_nonzero(is_pass & is_hom_alt & is_GQ_30)
    results['num_GQ_99']                = np.count_nonzero(is_pass & is_called & is_GQ_99)
    results['num_het_GQ_99']            = np.count_nonzero(is_pass & is_het & is_GQ_99)
    results['num_hom_alt_GQ_99']        = np.count_nonzero(is_pass & is_hom_alt & is_GQ_99)

    results['pc_pass']                  = 0.0 if results['num_called'] == 0 else \
        results['num_pass_called'] / results['num_called']
    results['pc_missing']               = 0.0 if results['num_variants'] == 0 else \
        results['num_missing'] / results['num_variants']
    results['pc_pass_missing']          = 0.0 if results['num_pass_variants'] == 0 else \
        results['num_pass_missing'] / results['num_pass_variants']
    results['pc_het']                   = 0.0 if results['num_called'] == 0 else \
        results['num_het'] / results['num_called']
    results['pc_pass_het']              = 0.0 if results['num_pass_called'] == 0 else \
        results['num_pass_het'] / results['num_pass_called']
    results['pc_hq_snp_het']            = 0.0 if results['num_hq_snp_called'] == 0 else \
        results['num_hq_snp_het'] / results['num_hq_snp_called']
    results['pc_vhq_snp_het']           = 0.0 if results['num_vhq_snp_called'] == 0 else \
        results['num_vhq_snp_het'] / results['num_vhq_snp_called']
    results['pc_hom_alt']               = 0.0 if results['num_called'] == 0 else \
        results['num_hom_alt'] / results['num_called']
    results['pc_pass_hom_alt']          = 0.0 if results['num_pass_called'] == 0 else \
        results['num_pass_hom_alt'] / results['num_pass_called']
    results['pc_snp']                   = 0.0 if results['num_pass_non_ref'] == 0 else \
        (results['num_snp_het'] + results['num_snp_hom_alt']) / results['num_pass_non_ref']
#     results['pc_snp_v2']                = (results['num_snp_het'] + results['num_snp_hom_alt']) / (results['num_snp_het'] + results['num_snp_hom_alt'] + results['num_indel_het'] + results['num_indel_hom_alt'])
    results['pc_biallelic']             = 0.0 if results['num_pass_non_ref'] == 0 else \
        (results['num_biallelic_het'] + results['num_biallelic_hom_alt']) / results['num_pass_non_ref']
    results['pc_spanning_del']          = 0.0 if results['num_pass_non_ref'] == 0 else \
        (results['num_spanning_del_het'] + results['num_spanning_del_hom_alt']) / results['num_pass_non_ref']
    results['pc_mutliallelic']          = 0.0 if results['num_pass_non_ref'] == 0 else \
        (results['num_multiallelic_het'] + results['num_multiallelic_hom_alt']) / results['num_pass_non_ref']
    results['pc_ins']                   = 0.0 if (results['num_ins'] + results['num_del']) == 0 else \
        (results['num_ins'] / (results['num_ins'] + results['num_del']))
    results['pc_coding']                = 0.0 if results['num_pass_non_ref'] == 0 else \
        results['num_coding'] / results['num_pass_non_ref']
#     results['pc_bi_nonsynonymous']      = results['num_bi_nonsynonymous'] / (results['num_biallelic_het'] + results['num_biallelic_hom_alt'])
    results['pc_bi_nonsynonymous']      = 0.0 if (results['num_bi_nonsynonymous'] + results['num_bi_synonymous']) == 0 else \
        results['num_bi_nonsynonymous'] / (results['num_bi_nonsynonymous'] + results['num_bi_synonymous'])
    results['pc_frameshift']            = 0.0 if (results['num_frameshift'] + results['num_inframe']) == 0 else \
        results['num_frameshift'] / (results['num_frameshift'] + results['num_inframe'])
    results['pc_bi_frameshift']         = 0.0 if (results['num_bi_frameshift'] + results['num_bi_inframe']) == 0 else \
        results['num_bi_frameshift'] / (results['num_bi_frameshift'] + results['num_bi_inframe'])
    results['pc_hq_frameshift']         = 0.0 if (results['num_hq_frameshift'] + results['num_hq_inframe']) == 0 else \
        results['num_hq_frameshift'] / (results['num_hq_frameshift'] + results['num_hq_inframe'])
    results['pc_bi_frameshift_snpeff']  = 0.0 if (results['num_bi_frameshift_snpeff'] + results['num_bi_inframe_snpeff']) == 0 else \
        results['num_bi_frameshift_snpeff'] / (results['num_bi_frameshift_snpeff'] + results['num_bi_inframe_snpeff'])
    results['pc_bi_transition']         = 0.0 if (results['num_bi_transition'] + results['num_bi_transversion']) == 0 else \
        results['num_bi_transition'] / (results['num_bi_transition'] + results['num_bi_transversion'])
    results['pc_bi_AT_to_AT']           = 0.0 if (results['num_bi_AT_to_AT'] + results['num_bi_CG_to_CG'] + results['num_bi_AT_to_CG'] + results['num_bi_CG_to_AT']) == 0 else \
        results['num_bi_AT_to_AT'] / (results['num_bi_AT_to_AT'] + results['num_bi_CG_to_CG'] + results['num_bi_AT_to_CG'] + results['num_bi_CG_to_AT'])
    results['pc_bi_CG_to_CG']           = 0.0 if (results['num_bi_AT_to_AT'] + results['num_bi_CG_to_CG'] + results['num_bi_AT_to_CG'] + results['num_bi_CG_to_AT']) == 0 else \
        results['num_bi_CG_to_CG'] / (results['num_bi_AT_to_AT'] + results['num_bi_CG_to_CG'] + results['num_bi_AT_to_CG'] + results['num_bi_CG_to_AT'])
    results['pc_bi_AT_to_CG']           = 0.0 if (results['num_bi_AT_to_AT'] + results['num_bi_CG_to_CG'] + results['num_bi_AT_to_CG'] + results['num_bi_CG_to_AT']) == 0 else \
        results['num_bi_AT_to_CG'] / (results['num_bi_AT_to_AT'] + results['num_bi_CG_to_CG'] + results['num_bi_AT_to_CG'] + results['num_bi_CG_to_AT'])
    results['pc_bi_CG_to_AT']           = 0.0 if (results['num_bi_AT_to_AT'] + results['num_bi_CG_to_CG'] + results['num_bi_AT_to_CG'] + results['num_bi_CG_to_AT']) == 0 else \
        results['num_bi_CG_to_AT'] / (results['num_bi_AT_to_AT'] + results['num_bi_CG_to_CG'] + results['num_bi_AT_to_CG'] + results['num_bi_CG_to_AT'])
    results['pc_phased']                = 0.0 if results['num_pass_non_ref'] == 0 else \
        results['num_phased_non_ref'] / results['num_pass_non_ref']
    results['pc_phased_hom_ref']        = 0.0 if results['num_phased'] == 0 else \
        results['num_phased_hom_ref'] / results['num_phased']
    results['pc_phased_missing']        = 0.0 if results['num_phased'] == 0 else \
        results['num_phased_missing'] / results['num_phased']
    results['pc_GQ_30']                 = 0.0 if results['num_pass_called'] == 0 else \
        results['num_GQ_30'] / results['num_pass_called']
    results['pc_het_GQ_30']             = 0.0 if results['num_pass_het'] == 0 else \
        results['num_het_GQ_30'] / results['num_pass_het']
    results['pc_hom_alt_GQ_30']         = 0.0 if results['num_pass_hom_alt'] == 0 else \
        results['num_hom_alt_GQ_30'] / results['num_pass_hom_alt']
    results['pc_GQ_99']                 = 0.0 if results['num_pass_called'] == 0 else \
        results['num_GQ_99'] / results['num_pass_called']
    results['pc_het_GQ_99']             = 0.0 if results['num_pass_het'] == 0 else \
        results['num_het_GQ_99'] / results['num_pass_het']
    results['pc_hom_alt_GQ_99']         = 0.0 if results['num_pass_hom_alt'] == 0 else \
        results['num_hom_alt_GQ_99'] / results['num_pass_hom_alt']
     
    results['mean_GQ']                  = np.mean(GQ[is_pass])
    results['mean_GQ_hom_ref']          = np.mean(GQ[is_pass & is_hom_ref])
    results['mean_GQ_het']              = np.mean(GQ[is_pass & is_het])
    results['mean_GQ_hom_alt']          = np.mean(GQ[is_pass & is_hom_alt])
    results['mean_DP']                  = np.mean(DP[is_pass])
    results['mean_DP_hom_ref']          = np.mean(DP[is_pass & is_hom_ref])
    results['mean_DP_het']              = np.mean(DP[is_pass & is_het])
    results['mean_DP_hom_alt']          = np.mean(DP[is_pass & is_hom_alt])
#     results['mean_GQ_2']                = np.nanmean(GQ[is_pass])
#     results['mean_DP_2']                = np.nanmean(DP[is_pass])
#     results['mean_DP']                  = np.mean(DP)
#     results['mean_DP_2']                = np.nanmean(DP)

    results['mean_indel_len']           = np.mean(indel_len[is_pass])
    results['total_indel_len']          = np.sum(indel_len[is_pass])

    print('\t'.join([str(x) for x in list(results.keys())]))
    print('\t'.join([str(x) for x in list(results.values())]))

#     return(results, is_pass, is_phased, is_non_ref, is_hom_ref, is_missing)
#     return(results, svlen, svlen1, svlen2, indel_len, is_indel, is_inframe, is_frameshift, is_pass, is_bi, is_non_ref, is_frameshift_snpeff, is_inframe_snpeff, is_coding, is_vqslod6)
#     return(results, is_pass, is_called, is_GQ_30)
    return(results)


- Create sample level summaries of genotypes
    - #Pass/Fail SNP/INDEL BI/MU/SD het/hom
    - Pass/Fail
    - SNP/INDEL
    - INS/DEL
    - Coding/non-coding
    - singletons
    - singleton mean VQSLOD
    - het/hom
    - NS/S
    - mean GQ
    - mean DP
    - %phased
    - num_pass_biallelic_coding_snp_het, num_VQSLOD6_biallelic_coding_snp_het
    - mean indel size, total_indel_size
    - %coding indel mod3
    - #transitions/transversions, Ti/Tv
    

In [302]:
results = create_sample_summary_2(index=275)

sample_id	num_variants	num_pass_variants	num_missing	num_pass_missing	num_called	num_pass_called	num_hom_ref	num_het	num_pass_het	num_hom_alt	num_pass_hom_alt	num_pass_non_ref	num_biallelic_het	num_biallelic_hom_alt	num_spanning_del_het	num_spanning_del_hom_alt	num_multiallelic_het	num_multiallelic_hom_alt	num_snp_het	num_snp_hom_alt	num_snp	num_indel_het	num_indel_hom_alt	num_indel	num_ins_het	num_ins_hom_alt	num_ins	num_del_het	num_del_hom_alt	num_del	num_coding_het	num_coding_hom_alt	num_coding	num_hq_snp_called	num_hq_snp_hom_ref	num_hq_snp_het	num_hq_snp_hom_alt	num_vhq_snp_called	num_vhq_snp_hom_ref	num_vhq_snp_het	num_vhq_snp_hom_alt	num_singleton	num_biallelic_singleton	num_hq_snp_singleton	num_vhq_snp_singleton	num_bi_nonsynonymous	num_bi_synonymous	num_frameshift	num_inframe	num_bi_frameshift	num_bi_inframe	num_hq_frameshift	num_hq_inframe	num_bi_frameshift_snpeff	num_bi_inframe_snpeff	num_bi_transition	num_bi_transversion	num_bi_AT_to_AT	num_bi_CG_to_CG	num_bi_AT_to_CG	num_b

In [277]:
results, is_pass, is_called, is_GQ_30 = create_sample_summary_2()

sample_id	num_variants	num_pass_variants	num_missing	num_pass_missing	num_called_2	num_called	num_pass_called_3	num_pass_called_2	num_pass_called	num_hom_ref	num_het	num_pass_het	num_hom_alt	num_pass_hom_alt	num_pass_non_ref	num_variants_2	num_biallelic_het	num_biallelic_hom_alt	num_spanning_del_het	num_spanning_del_hom_alt	num_multiallelic_het	num_multiallelic_hom_alt	num_snp_het	num_snp_hom_alt	num_snp	num_indel_het	num_indel_hom_alt	num_indel	num_ins_het	num_ins_hom_alt	num_ins	num_del_het	num_del_hom_alt	num_del	num_coding_het	num_coding_hom_alt	num_coding	num_hq_snp_called	num_hq_snp_hom_ref	num_hq_snp_het	num_hq_snp_hom_alt	num_vhq_snp_called	num_vhq_snp_hom_ref	num_vhq_snp_het	num_vhq_snp_hom_alt	num_singleton	num_biallelic_singleton	num_hq_snp_singleton	num_vhq_snp_singleton	num_bi_nonsynonymous	num_bi_synonymous	num_frameshift	num_inframe	num_frameshift_2	num_inframe_2	num_bi_frameshift	num_bi_inframe	num_hq_frameshift	num_hq_inframe	num_bi_frameshift_snpeff	num_bi_inframe_snp

In [279]:
print('pc_GQ_30', results['pc_GQ_30'])
print('pc_het_GQ_30', results['pc_het_GQ_30'])
print('pc_hom_alt_GQ_30', results['pc_hom_alt_GQ_30'])
print('pc_GQ_99', results['pc_GQ_99'])
print('pc_het_GQ_99', results['pc_het_GQ_99'])
print('pc_hom_alt_GQ_99', results['pc_hom_alt_GQ_99'])


pc_GQ_30 0.723831988412994
pc_het_GQ_30 0.7071468730363462
pc_hom_alt_GQ_30 0.43987599183510595
pc_GQ_99 0.41224877577763985
pc_het_GQ_99 0.3885967523233125
pc_hom_alt_GQ_99 0.1302546157742069


In [ ]:
results = create_sample_summary_2()

In [263]:
print('num_frameshift', results['num_frameshift'])
print('num_inframe', results['num_inframe'])
print('num_bi_frameshift', results['num_bi_frameshift'])
print('num_bi_inframe', results['num_bi_inframe'])
print('num_hq_frameshift', results['num_bi_frameshift'])
print('num_hq_inframe', results['num_bi_inframe'])
print('num_bi_frameshift_snpeff', results['num_bi_frameshift_snpeff'])
print('num_bi_inframe_snpeff', results['num_bi_inframe_snpeff'])
print()
print('pc_frameshift', results['pc_frameshift'])
print('pc_bi_frameshift', results['pc_bi_frameshift'])
print('pc_hq_frameshift', results['pc_bi_frameshift'])
print('pc_bi_frameshift_snpeff', results['pc_bi_frameshift_snpeff'])



num_frameshift 6589
num_inframe 7780
num_bi_frameshift 39
num_bi_inframe 62
num_hq_frameshift 39
num_hq_inframe 62
num_bi_frameshift_snpeff 32
num_bi_inframe_snpeff 62

pc_frameshift 0.4585566149349294
pc_bi_frameshift 0.38613861386138615
pc_hq_frameshift 0.38613861386138615
pc_bi_frameshift_snpeff 0.3404255319148936


In [254]:
np.count_nonzero(is_pass & is_bi & is_indel & is_non_ref & is_coding & is_frameshift)

39

In [264]:
np.where(is_pass & is_vqslod6 & is_bi & is_indel & is_non_ref & is_coding & is_frameshift)[0]

array([], dtype=int64)

In [253]:
np.where(is_pass & is_bi & is_indel & is_non_ref & is_coding & is_frameshift)[0]

array([ 140202,  144157,  152034,  152668,  336378,  421546,  923700,
       1064264, 1222585, 1232245, 1232248, 1244211, 1391912, 1812400,
       1883953, 1883956, 2062801, 2230731, 2508059, 2907499, 2930134,
       3102859, 3216702, 3344070, 3344071, 3640329, 3764947, 4841378,
       5465200, 5479183, 5509733, 5905061, 6009449, 6009450, 6010092,
       6010093, 6025048, 6025161, 6025589])

In [255]:
np.where(is_pass & is_bi & is_indel & is_non_ref & is_coding & is_frameshift_snpeff)[0]

array([ 144157,  152034,  152668,  421546,  923700, 1064264, 1222585,
       1232245, 1232248, 1244211, 1812400, 1883953, 1883956, 2062801,
       2508059, 2907499, 2930134, 3102859, 3216702, 3344070, 3344071,
       3640329, 3764947, 4841378, 5465200, 5479183, 5509733, 5905061,
       6009449, 6009450, 6010092, 6010093])

In [256]:
np.setdiff1d(
    np.where(is_pass & is_bi & is_indel & is_non_ref & is_coding & is_frameshift)[0],
    np.where(is_pass & is_bi & is_indel & is_non_ref & is_coding & is_frameshift_snpeff)[0]
)

array([ 140202,  336378, 1391912, 2230731, 6025048, 6025161, 6025589])

In [258]:
np.setdiff1d(
    np.where(is_pass & is_bi & is_indel & is_non_ref & is_coding & is_frameshift_snpeff)[0],
    np.where(is_pass & is_bi & is_indel & is_non_ref & is_coding & is_frameshift)[0]
)

array([], dtype=int64)

In [259]:
index=140202
print(hdf['Pf60']['variants']['REF'][index])
print(hdf['Pf60']['variants']['ALT'][index, :])
print(hdf['Pf60']['variants']['CDS'][index])
print(hdf['Pf60']['variants']['SNPEFF_EFFECT'][index])
print(hdf['Pf60']['calldata']['genotype'][index, 0, :])
print(hdf['Pf60']['calldata']['AD'][index, 0, :])
print(hdf['Pf60']['calldata']['PL'][index, 0, :])
print(hdf['Pf60']['calldata']['PID'][index, 0])
print(hdf['Pf60']['calldata']['PGT'][index, 0])


b'TA'
[b'T' b'' b'' b'' b'' b'']
True
b''
[1 1]
[0 9 0 0 0 0 0]
[310  27   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]
b'.'
b'.'


In [242]:
index=61186
print(hdf['Pf60']['variants']['REF'][index])
print(hdf['Pf60']['variants']['ALT'][index, :])
print(hdf['Pf60']['variants']['SNPEFF_EFFECT'][index])
print(hdf['Pf60']['calldata']['genotype'][index, 0, :])
print(hdf['Pf60']['calldata']['AD'][index, 0, :])
print(hdf['Pf60']['calldata']['PL'][index, 0, :])
print(hdf['Pf60']['calldata']['PID'][index, 0])
print(hdf['Pf60']['calldata']['PGT'][index, 0])


b'TTA'
[b'T' b'' b'' b'' b'' b'']
b'INTERGENIC'
[1 1]
[ 0 10  0  0  0  0  0]
[427  30   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]
b'.'
b'.'


In [190]:
is_phased_hom_ref = (is_pass & is_phased & is_hom_ref)
is_phased_missing = (is_pass & is_phased & is_missing)
print(np.count_nonzero(is_phased_hom_ref))
print(np.count_nonzero(is_phased_missing))


2953
87


In [191]:
np.where(is_phased_hom_ref)

(array([  71926,   73593,   82144, ..., 5997095, 6000998, 6016847]),)

In [208]:
index=71926
print(hdf['Pf60']['variants']['REF'][index])
print(hdf['Pf60']['variants']['ALT'][index, :])
print(hdf['Pf60']['calldata']['genotype'][index, 0, :])
print(hdf['Pf60']['calldata']['AD'][index, 0, :])
print(hdf['Pf60']['calldata']['PL'][index, 0, :])
print(hdf['Pf60']['calldata']['PID'][index, 0])
print(hdf['Pf60']['calldata']['PGT'][index, 0])


b'ATTTATATTATCATCCCCTTTTATATTATCACCCCCTTTTATATTATCACCCCCTTTTATATTATCATCC'
[b'TTTTATATTATCATCCCCTTTTATATTATCACCCCCTTTTATATTATCACCCCCTTTTATATTATCATCC'
 b'ATATTATCATCCCCTTTTATATTATCATCCCCTTTTATATTATCACCCCCTTTTATATTATCACCCCCTTTTATATTATCATCC'
 b'ATATTATCACCCCCTTTTATATTATCATCCCCTTTTATATTATCACCCCCTTTTATATTATCACCCCCTTTTATATTATCATCC'
 b'*'
 b'ATATTATCACCCCCTTTTATATTATCATCCCCTTTTATATTATCATCCCCTTTTATATTATCACCCCCTTTTATATTATCACCCCCTTTTATATTATCATCC'
 b'ATATTATCATCCCCTTTTATATTATCACCCCCTTTTATATTATCATCCCCTTTTATATTATCACCCCCTTTTATATTATCACCCCCTTTTATATTATCATCC']
[0 0]
[38  3  0  0  0  0  0]
[   0   47 1388  124 1397 1473  124 1397 1473 1473  124 1397 1473 1473 1473
  124 1397 1473 1473 1473 1473  124 1397 1473 1473 1473 1473 1473]
b'148912_TCCCC'
b'1|0'


In [209]:
index=79318
print(hdf['Pf60']['variants']['REF'][index])
print(hdf['Pf60']['variants']['ALT'][index, :])
print(hdf['Pf60']['calldata']['genotype'][index, 0, :])
print(hdf['Pf60']['calldata']['AD'][index, 0, :])
print(hdf['Pf60']['calldata']['PL'][index, 0, :])
print(hdf['Pf60']['calldata']['PID'][index, 0])
print(hdf['Pf60']['calldata']['PGT'][index, 0])


b'CATATATATATATATATAT'
[b'CAT' b'CATAT' b'CATATAT' b'CATATATAT' b'C' b'CATATATATAT']
[-1 -1]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
b'181835_CAT_C'
b'0|1'


In [207]:
index=79318
print(hdf['Pf60']['variants']['REF'][71926])
print(hdf['Pf60']['variants']['ALT'][71926, :])
print(hdf['Pf60']['calldata']['genotype'][71926, 0, :])
print(hdf['Pf60']['calldata']['AD'][71926, 0, :])
print(hdf['Pf60']['calldata']['PL'][71926, 0, :])
print(hdf['Pf60']['calldata']['PID'][71926, 0])
print(hdf['Pf60']['calldata']['PGT'][71926, 0])


b'ATTTATATTATCATCCCCTTTTATATTATCACCCCCTTTTATATTATCACCCCCTTTTATATTATCATCC'
[b'TTTTATATTATCATCCCCTTTTATATTATCACCCCCTTTTATATTATCACCCCCTTTTATATTATCATCC'
 b'ATATTATCATCCCCTTTTATATTATCATCCCCTTTTATATTATCACCCCCTTTTATATTATCACCCCCTTTTATATTATCATCC'
 b'ATATTATCACCCCCTTTTATATTATCATCCCCTTTTATATTATCACCCCCTTTTATATTATCACCCCCTTTTATATTATCATCC'
 b'*'
 b'ATATTATCACCCCCTTTTATATTATCATCCCCTTTTATATTATCATCCCCTTTTATATTATCACCCCCTTTTATATTATCACCCCCTTTTATATTATCATCC'
 b'ATATTATCATCCCCTTTTATATTATCACCCCCTTTTATATTATCATCCCCTTTTATATTATCACCCCCTTTTATATTATCACCCCCTTTTATATTATCATCC']
[0 0]
[38  3  0  0  0  0  0]
[   0   47 1388  124 1397 1473  124 1397 1473 1473  124 1397 1473 1473 1473
  124 1397 1473 1473 1473 1473  124 1397 1473 1473 1473 1473 1473]
b'148912_TCCCC'
b'1|0'


In [194]:
hdf['Pf60']['variants']['ALT'][:][71926, :]

array([b'TTTTATATTATCATCCCCTTTTATATTATCACCCCCTTTTATATTATCACCCCCTTTTATATTATCATCC',
       b'ATATTATCATCCCCTTTTATATTATCATCCCCTTTTATATTATCACCCCCTTTTATATTATCACCCCCTTTTATATTATCATCC',
       b'ATATTATCACCCCCTTTTATATTATCATCCCCTTTTATATTATCACCCCCTTTTATATTATCACCCCCTTTTATATTATCATCC',
       b'*',
       b'ATATTATCACCCCCTTTTATATTATCATCCCCTTTTATATTATCATCCCCTTTTATATTATCACCCCCTTTTATATTATCACCCCCTTTTATATTATCATCC',
       b'ATATTATCATCCCCTTTTATATTATCACCCCCTTTTATATTATCATCCCCTTTTATATTATCACCCCCTTTTATATTATCACCCCCTTTTATATTATCATCC'], 
      dtype='|S600')

In [195]:
hdf['Pf60']['calldata']['genotype'][:, 0, :][71926, :]

array([0, 0], dtype=int8)

In [196]:
hdf['Pf60']['calldata']['genotype'][71926, 0, :]

array([0, 0], dtype=int8)

In [197]:
hdf['Pf60']['calldata']['AD'][71926, 0, :]

array([38,  3,  0,  0,  0,  0,  0], dtype=uint16)

In [199]:
hdf['Pf60']['calldata']['PL'][71926, 0, :]

array([   0,   47, 1388,  124, 1397, 1473,  124, 1397, 1473, 1473,  124,
       1397, 1473, 1473, 1473,  124, 1397, 1473, 1473, 1473, 1473,  124,
       1397, 1473, 1473, 1473, 1473, 1473], dtype=uint16)

In [201]:
hdf['Pf60']['calldata']['PID'][71926, 0]

b'148912_TCCCC'

In [202]:
hdf['Pf60']['calldata']['PGT'][71926, 0]

b'1|0'

In [192]:
np.where(is_phased_missing)

(array([  79318,   98016,   98017,   98018,   98019,   98020,  335358,
         367251,  367256,  367261,  379392,  529130,  529131,  529132,
         529133,  529134,  534104,  548215,  608215,  623212,  824808,
         873459,  958601,  958602,  977466,  983633,  984468, 1069517,
        1183037, 1183038, 1183039, 1183040, 1183041, 1183042, 1183043,
        1214900, 1216035, 1287530, 1367360, 1479961, 1491183, 1988099,
        2017840, 2033643, 2112752, 2238039, 2249990, 2298154, 2379711,
        2395313, 2395314, 2432522, 2473649, 2644124, 2734966, 2860920,
        2879401, 2921722, 3215240, 3282816, 3586075, 3643032, 3698994,
        3732870, 3736603, 3736604, 3736605, 3742890, 3790927, 3876509,
        3899903, 4048092, 4082213, 4153866, 4300349, 4417022, 4470618,
        4659787, 4908652, 4936233, 5344427, 5430324, 5749911, 5796103,
        5796106, 5796114, 5913608]),)

In [184]:
pd.value_counts(PGT[:,0])

b'.'      3591768
b''       2373818
b'0|1'      48582
b'1|1'      34755
b'1|0'       2773
dtype: int64

In [168]:
results, mutations = create_sample_summary()

sample_id	num_variants	num_pass_variants	num_missing	num_pass_missing	num_called	num_pass_called	num_het	num_pass_het	num_hom_alt	num_pass_hom_alt	num_pass_non_ref	num_biallelic_het	num_biallelic_hom_alt	num_spanning_del_het	num_spanning_del_hom_alt	num_multiallelic_het	num_multiallelic_hom_alt	num_snp_hom_ref	num_snp_het	num_snp_hom_alt	num_indel_hom_ref	num_indel_het	num_indel_hom_alt	num_ins_het	num_ins_hom_alt	num_ins	num_del_het	num_del_hom_alt	num_del	num_coding_het	num_coding_hom_alt	num_coding	num_vhq_snp_hom_ref	num_vhq_snp_het	num_vhq_snp_hom_alt	num_singleton	num_biallelic_singleton	num_vhq_snp_singleton	num_bi_nonsynonymous	num_bi_synonymous	num_bi_frameshift	num_bi_inframe	num_bi_transition	num_bi_transversion	num_bi_AT_to_AT	num_bi_CG_to_CG	num_bi_AT_to_CG	num_bi_CG_to_AT	pc_pass	pc_missing	pc_pass_missing	pc_het	pc_pass_het	pc_hom_alt	pc_pass_hom_alt	pc_snp	pc_biallelic	pc_spanning_del	pc_mutliallelic	pc_ins	pc_coding	pc_bi_nonsynonymous	pc_bi_frameshift	pc_bi_transition

In [140]:
is_pass = hdf['Pf60']['variants']['FILTER_PASS'][:]
is_snp = (hdf['Pf60']['variants']['VARIANT_TYPE'][:][is_pass] == b'SNP')
is_bi = (hdf['Pf60']['variants']['MULTIALLELIC'][:][is_pass] == b'BI')
temp = hdf['Pf60']['variants']['REF'][:][is_pass][is_snp]

In [141]:
mutations = np.char.add(
    hdf['Pf60']['variants']['REF'][:][is_pass][(is_snp & is_bi)],
    hdf['Pf60']['variants']['ALT'][:, 0][is_pass][(is_snp & is_bi)]
)


In [142]:
pd.value_counts(mutations)

b'TC'    256014
b'AG'    255456
b'GA'    191133
b'CT'    189628
b'AT'    181723
b'TA'    181298
b'GT'     75142
b'CA'     74347
b'TG'     71297
b'AC'     70298
b'GC'     40383
b'CG'     40167
dtype: int64

In [138]:
pd.value_counts(mutations)

b'TC'    287482
b'AG'    286027
b'GA'    233352
b'CT'    230504
b'TA'    222426
b'AT'    210870
b'GT'    103832
b'CA'    103091
b'T*'     92997
b'TG'     83774
b'A*'     83332
b'AC'     82614
b'GC'     56907
b'CG'     56619
b'G*'     22504
b'C*'     20979
dtype: int64

In [133]:
list(hdf['Pf60'].keys())

['calldata', 'samples', 'variants']

In [110]:
pd.value_counts(hdf['Pf60']['variants']['AC'][:,0] == 0)

True     4664914
False    1386782
dtype: int64

In [119]:
pd.value_counts(hdf['Pf60']['variants']['SNPEFF_EFFECT'][:])

b'INTERGENIC'                           2795496
b'NON_SYNONYMOUS_CODING'                1430363
b'SYNONYMOUS_CODING'                     641017
b'INTRON'                                449418
b'FRAME_SHIFT'                           444677
b'INTRAGENIC'                             89670
b''                                       70597
b'CODON_INSERTION'                        47470
b'CODON_DELETION'                         27491
b'CODON_CHANGE_PLUS_CODON_DELETION'       17675
b'STOP_GAINED'                            17488
b'CODON_CHANGE_PLUS_CODON_INSERTION'      15638
b'SPLICE_SITE_DONOR'                       1034
b'SPLICE_SITE_ACCEPTOR'                    1017
b'STOP_LOST'                                943
b'SYNONYMOUS_STOP'                          877
b'START_LOST'                               653
b'EXON_DELETED'                              93
b'NON_SYNONYMOUS_START'                      79
dtype: int64

In [154]:
7182/20

359.1

In [159]:
pd.value_counts(hdf['Pf60']['variants']['SNPEFF_EFFECT'][:][(
            hdf['Pf60']['variants']['FILTER_PASS'][:] &
            (hdf['Pf60']['variants']['MULTIALLELIC'][:] == b'BI') &
            (hdf['Pf60']['variants']['AC'][:, 0] > 359) &
            (hdf['Pf60']['variants']['AC'][:, 0] < (7182-359))
        )])

b'NON_SYNONYMOUS_CODING'                8240
b'INTERGENIC'                           7939
b'SYNONYMOUS_CODING'                    3709
b'INTRON'                                956
b'INTRAGENIC'                            808
b'FRAME_SHIFT'                            88
b''                                       83
b'CODON_INSERTION'                        77
b'CODON_DELETION'                         53
b'STOP_GAINED'                            31
b'CODON_CHANGE_PLUS_CODON_DELETION'       24
b'CODON_CHANGE_PLUS_CODON_INSERTION'      11
b'SYNONYMOUS_STOP'                         4
b'SPLICE_SITE_ACCEPTOR'                    3
b'STOP_LOST'                               3
b'SPLICE_SITE_DONOR'                       2
b'START_LOST'                              1
dtype: int64

In [162]:
print('num_pass_non_ref', results['num_pass_non_ref'])
print('num_pass_non_ref_2', results['num_pass_non_ref_2'])


num_pass_non_ref 95394
num_pass_non_ref_2 95394


In [147]:
print('pc_bi_transition', results['pc_bi_transition'])
print('pc_bi_frameshift', results['pc_bi_frameshift'])
print('num_bi_frameshift', results['num_bi_frameshift'])
print('num_bi_inframe', results['num_bi_inframe'])
print('pc_biallelic', results['pc_biallelic'])
print('pc_spanning_del', results['pc_spanning_del'])
print('pc_mutliallelic', results['pc_mutliallelic'])
print('pc_bi_nonsynonymous', results['pc_bi_nonsynonymous'])
print('pc_bi_nonsynonymous_2', results['pc_bi_nonsynonymous_2'])


pc_bi_transition 0.5484287159641179
pc_bi_frameshift 0.4552585868773463
num_bi_frameshift 8125
num_bi_inframe 9722
pc_biallelic 0.0946810071912
pc_spanning_del 0.427815166572
pc_mutliallelic 0.477503826236
pc_bi_nonsynonymous 76.7018379097
pc_bi_nonsynonymous_2 0.6681329300707124


In [ ]:
pd.value_counts(mutations)

In [93]:
print('num_snp_hom_ref', results['num_snp_hom_ref'])
print('num_snp_het', results['num_snp_het'])
print('num_snp_hom_alt', results['num_snp_hom_alt'])
print('num_indel_hom_ref', results['num_indel_hom_ref'])
print('num_indel_het', results['num_indel_het'])
print('num_indel_hom_alt', results['num_indel_hom_alt'])
print()
print('pc_snp', results['pc_snp'])
# print('pc_snp_v2', results['pc_snp_v2'])


num_snp_hom_ref 2094500
num_snp_het 9188
num_snp_hom_alt 19951
num_indel_hom_ref 709906
num_indel_het 21049
num_indel_hom_alt 45206

pc_snp 0.305459462859
pc_snp_v2 0.305459462859


In [51]:
create_sample_summary()

sample_id	num_variants	num_pass_variants	num_missing	num_pass_missing	num_called	num_pass_called	num_het	num_pass_het	num_hom_alt	num_pass_hom_alt	num_snp_hom_ref	num_snp_het	num_snp_hom_alt	num_indel_hom_ref	num_indel_het	num_indel_hom_alt	num_ins_hom_ref	num_ins_het	num_ins	num_del_hom_ref	num_del_het	num_del	pc_pass	pc_missing	pc_pass_missing	pc_het	pc_pass_het	pc_hom_alt	pc_pass_hom_alt	pc_snp	pc_ins
FP0008-C	6051696	3114760	705552	214960	5346144	2899800	75703	30237	97032	65157	2094500	9188	19951	709906	21049	45206	0	4166	4166	0	13248	13248	0.542409632064	0.116587482253	0.0690133429221	0.0141602994607	0.0104272708463	0.0181499039308	0.0224694806538	0.305459462859	0.239232801194


In [53]:
!/nfs/team112_internal/rp7/src/github/malariagen/methods-dev/builds/Pf\ 6.0/scripts/create_sample_summary.py

sample_id	num_variants	num_pass_variants	num_missing	num_pass_missing	num_called	num_pass_called	num_het	num_pass_het	num_hom_alt	num_pass_hom_alt	num_snp_hom_ref	num_snp_het	num_snp_hom_alt	num_indel_hom_ref	num_indel_het	num_indel_hom_alt	num_ins_hom_ref	num_ins_het	num_ins	num_del_hom_ref	num_del_het	num_del	pc_pass	pc_missing	pc_pass_missing	pc_het	pc_pass_het	pc_hom_alt	pc_pass_hom_alt	pc_snp	pc_ins
FP0008-C	6051696	3114760	705552	214960	5346144	2899800	75703	30237	97032	65157	2094500	9188	19951	709906	21049	45206	0	4166	4166	0	13248	13248	0.542409632064	0.116587482253	0.0690133429221	0.0141602994607	0.0104272708463	0.0181499039308	0.0224694806538	0.305459462859	0.239232801194


In [61]:
fo = open(run_create_sample_summary_job_fn, 'w')
print('''HDF5_FN=$1
LSB_JOBINDEX=1
INDEX=$((LSB_JOBINDEX-1))
echo $INDEX

/nfs/team112_internal/rp7/src/github/malariagen/methods-dev/builds/Pf\ 6.0/scripts/create_sample_summary.py \
--hdf5_fn $HDF5_FN --index $INDEX

''', file=fo)
fo.close()


In [265]:
fo = open(run_create_sample_summary_job_fn, 'w')
print('''HDF5_FN=$1
RELEASE=$2
# LSB_JOBINDEX=1
INDEX=$((LSB_JOBINDEX-1))
echo $INDEX
OUTPUT_FN=%s/sample_summaries/$RELEASE/results_$INDEX.txt

/nfs/team112_internal/rp7/src/github/malariagen/methods-dev/builds/Pf\ 6.0/scripts/create_sample_summary.py \
--hdf5_fn $HDF5_FN --index $INDEX > $OUTPUT_FN

''' % (output_dir), file=fo)
fo.close()


In [98]:
!LSB_JOBINDEX=2 && bash {run_create_sample_summary_job_fn} \
/lustre/scratch111/malaria/rp7/data/methods-dev/builds/Pf6.0/20161128_HDF5_build/hdf5/Pf_60_npy_PID_a12.h5 \
Pf60

1


In [306]:
MEMORY=8000
# Kick off Pf 6.0 jobs
!bsub -q normal -G malaria-dk -J "summ[1-7182]" -n4 \
-R"select[mem>{MEMORY}] rusage[mem={MEMORY}] span[hosts=1]" -M {MEMORY} \
-o {"%s/log/output_%%J-%%I.log" % output_dir} bash {run_create_sample_summary_job_fn} \
/lustre/scratch111/malaria/rp7/data/methods-dev/builds/Pf6.0/20161128_HDF5_build/hdf5/Pf_60_npy_PID_a12.h5 \
Pf60

Job <9881744> is submitted to queue <normal>.


In [304]:
MEMORY=8000
# Kick off Pv 3.0 jobs
!bsub -q normal -G malaria-dk -J "summ[1-1001]" -n4 \
-R"select[mem>{MEMORY}] rusage[mem={MEMORY}] span[hosts=1]" -M {MEMORY} \
-o {"%s/log/output_%%J-%%I.log" % output_dir} bash {run_create_sample_summary_job_fn} \
{hdf_fn['Pv30']} \
Pv30

Job <9877241> is submitted to queue <normal>.


In [308]:
!(head -n 1 {output_dir}/sample_summaries/Pf60/results_0.txt & \
  cat {output_dir}/sample_summaries/Pf60/results_*.txt | grep -v '^sample_id') > {output_dir}/pf_60_summaries.txt

In [309]:
!(head -n 1 {output_dir}/sample_summaries/Pv30/results_0.txt & \
  cat {output_dir}/sample_summaries/Pv30/results_*.txt | grep -v '^sample_id') > {output_dir}/pv_30_summaries.txt

In [284]:
output_dir

'/lustre/scratch109/malaria/rp7/data/methods-dev/builds/Pf6.0/20161205_sample_level_summaries'

In [52]:
create_sample_summary(1)

sample_id	num_variants	num_pass_variants	num_missing	num_pass_missing	num_called	num_pass_called	num_het	num_pass_het	num_hom_alt	num_pass_hom_alt	num_snp_hom_ref	num_snp_het	num_snp_hom_alt	num_indel_hom_ref	num_indel_het	num_indel_hom_alt	num_ins_hom_ref	num_ins_het	num_ins	num_del_hom_ref	num_del_het	num_del	pc_pass	pc_missing	pc_pass_missing	pc_het	pc_pass_het	pc_hom_alt	pc_pass_hom_alt	pc_snp	pc_ins
FP0009-C	6051696	3114760	459029	84770	5592667	3029990	80727	19396	139628	101341	2121375	2466	30860	787878	16930	70481	0	4137	4137	0	10711	10711	0.541779083217	0.0758512985451	0.0272155800126	0.0144344370941	0.00640134125855	0.0249662638594	0.0334459849702	0.27602143502	0.278623383621


In [48]:
df_sample_summary, results = create_sample_summary()

0
1
2
3
4
5
6
7
8
9
10


In [43]:
df_sample_summary, results = create_sample_summary(index=100)

0
1
2
3
4
5
6
7
8
9
10


In [34]:
b'FP0008-C'.decode('ascii')

'FP0008-C'

In [44]:
'\t'.join([str(x) for x in list(results.values())])

'PA0027-C\t6051696\t3114760\t314186\t24651\t5737510\t3090109\t166376\t93930\t83656\t59078\t2123363\t24446\t18535\t2123363\t24446\t18535\t0\t14148\t14148\t0\t46527\t46527\t0.538580150623\t0.0519170163207\t0.00791425342563\t0.0289979451016\t0.0303969859963\t0.0145805410361\t0.0191184194473\t0.5\t0.233176761434'

In [40]:
'\t'.join([str(x) for x in list(results.values())])

'FP0008-C\t6051696\t3114760\t705552\t214960\t5346144\t2899800\t75703\t30237\t97032\t65157\t2094500\t9188\t19951\t2094500\t9188\t19951\t0\t4166\t4166\t0\t13248\t13248\t0.542409632064\t0.116587482253\t0.0690133429221\t0.0141602994607\t0.0104272708463\t0.0181499039308\t0.0224694806538\t0.5\t0.239232801194'

In [39]:
'\t'.join(list(results.values()))

TypeError: sequence item 1: expected str instance, int found

In [38]:
list(results.values())

['FP0008-C',
 6051696,
 3114760,
 705552,
 214960,
 5346144,
 2899800,
 75703,
 30237,
 97032,
 65157,
 2094500,
 9188,
 19951,
 2094500,
 9188,
 19951,
 0,
 4166,
 4166,
 0,
 13248,
 13248,
 0.54240963206378279,
 0.11658748225290894,
 0.069013342922087093,
 0.01416029946069541,
 0.01042727084626526,
 0.01814990393075832,
 0.022469480653838195,
 0.5,
 0.23923280119444126]

In [23]:
list(results.values())[0]

b'FP0008-C'

In [5]:
hdf = h5py.File(hdf_fn['Pv30'], 'r')
hdf['calldata']['genotype'].shape

(4084419, 1001, 2)

In [6]:
svlen = hdf['variants']['svlen'][:]
svlen

array([[ 0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0],
       ..., 
       [ 2, -1,  4, 33,  0,  0],
       [ 3,  0,  0,  0,  0,  0],
       [ 5,  0,  0,  0,  0,  0]], dtype=int32)

In [7]:
genotype = hdf['calldata']['genotype'][:, 0, :]
genotype

array([[-1, -1],
       [-1, -1],
       [-1, -1],
       ..., 
       [-1, -1],
       [-1, -1],
       [-1, -1]], dtype=int8)

In [8]:
pd.value_counts(genotype[:,0])

-1    2146665
 0    1900766
 1      29579
 2       5550
 3       1207
 4        402
 5        187
 6         63
dtype: int64

In [9]:
print(genotype.shape)
print(svlen.shape)

(4084419, 2)
(4084419, 6)


In [43]:
svlen1 = svlen[np.arange(svlen.shape[0]), genotype[:, 0] - 1]
svlen1[np.in1d(genotype[:, 0], [-1, 0])] = 0

In [48]:
pd.Series(svlen1).describe()

count    4.084419e+06
mean    -2.211991e-02
std      1.022534e+00
min     -1.500000e+02
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      8.700000e+01
dtype: float64

In [42]:
pd.value_counts(svlen1)

 0      4043181
-1         2741
-2         2601
 2         2302
-3         2191
 1         1865
 3         1770
-4         1741
 4         1377
-5         1047
-6          888
 5          875
 6          775
 12         598
-12         563
-8          560
 9          539
 8          523
-9          514
-7          497
 36         493
 24         483
 7          472
-18         445
 18         436
-15         421
 15         409
-24         383
 10         370
 30         360
         ...   
 153          1
 152          1
 149          1
 141          1
 131          1
 258          1
 130          1
 128          1
 127          1
 122          1
 121          1
 113          1
 166          1
 167          1
 177          1
 180          1
 183          1
 187          1
 194          1
 195          1
 198          1
 205          1
 208          1
 209          1
 213          1
 221          1
 230          1
 239          1
 252          1
 188          1
dtype: int64

In [61]:
svlen2 = svlen[genotype]

In [63]:
svlen

array([[ 0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0],
       ..., 
       [ 2, -1,  4, 33,  0,  0],
       [ 3,  0,  0,  0,  0,  0],
       [ 5,  0,  0,  0,  0,  0]], dtype=int32)

In [65]:
genotype[:,0]

array([-1, -1, -1, ..., -1, -1, -1], dtype=int8)

In [66]:
svlen[:, genotype[:,0]]

MemoryError: 

In [15]:
np.take(svlen[0:100000], genotype[0:100000,0]-1, axis=1)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [21]:
alt_indexes = genotype[:, 0] - 1
alt_indexes[alt_indexes < 0] = 0

In [18]:
pd.value_counts(alt_indexes)

-2    2146665
-1    1900766
 0      29579
 1       5550
 2       1207
 3        402
 4        187
 5         63
dtype: int64

In [30]:
np.take(svlen[0:10000], alt_indexes[0:10000], axis=0).shape

(10000, 6)

In [33]:
svlen[0:1002]

array([[ 0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0],
       ..., 
       [ 2,  0,  0,  0,  0,  0],
       [ 0, -2,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0]], dtype=int32)

In [34]:
alt_indexes[0:1002]

array([0, 0, 0, ..., 0, 0, 0], dtype=int8)

In [35]:
np.take(svlen[0:1002], alt_indexes[0:1002], axis=0)

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ..., 
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int32)

In [40]:
svlen[0:1002][np.arange(1002), alt_indexes[0:1002]].shape

(1002,)

In [26]:
alt_indexes[0:10000].shape

(10000,)

In [29]:
svlen[0:10000].shape

(10000, 6)

In [71]:
svlen

array([[ 0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0],
       ..., 
       [ 2, -1,  4, 33,  0,  0],
       [ 3,  0,  0,  0,  0,  0],
       [ 5,  0,  0,  0,  0,  0]], dtype=int32)

In [62]:
print(svlen2.shape)
svlen2

(4084419, 2, 6)


array([[[5, 0, 0, 0, 0, 0],
        [5, 0, 0, 0, 0, 0]],

       [[5, 0, 0, 0, 0, 0],
        [5, 0, 0, 0, 0, 0]],

       [[5, 0, 0, 0, 0, 0],
        [5, 0, 0, 0, 0, 0]],

       ..., 
       [[5, 0, 0, 0, 0, 0],
        [5, 0, 0, 0, 0, 0]],

       [[5, 0, 0, 0, 0, 0],
        [5, 0, 0, 0, 0, 0]],

       [[5, 0, 0, 0, 0, 0],
        [5, 0, 0, 0, 0, 0]]], dtype=int32)

In [44]:
temp = hdf['calldata']['genotype'][:, [0], :]

In [51]:
temp2=allel.GenotypeArray(temp)
temp2

NameError: name 'allel' is not defined

In [45]:
temp.shape

(4084419, 1, 2)

In [25]:
%%time
df_sample_summary = collections.OrderedDict()
# for release in genotypes_subset:
for release in ['Pv30', 'Pf60']:
    print(release)
    samples = hdf[release]['samples'][:]
    pass_variants = hdf[release]['variants']['FILTER_PASS'][:]
    
    print(0)
    is_snp = (hdf[release]['variants']['VARIANT_TYPE'][:][pass_variants] == b'SNP')
    is_bi = (hdf[release]['variants']['MULTIALLELIC'][:][pass_variants] == b'BI')
    is_sd = (hdf[release]['variants']['MULTIALLELIC'][:][pass_variants] == b'SD')
    is_mu = (hdf[release]['variants']['MULTIALLELIC'][:][pass_variants] == b'MU')
    is_ins = (hdf[release]['variants']['svlen'][:][pass_variants] > 0)
    is_del = (hdf[release]['variants']['svlen'][:][pass_variants] < 0)
    
    print(1)
    num_variants = genotypes_subset[release]['all'].shape[0]
    num_pass_variants = genotypes_subset[release]['pass'].shape[0]
    num_missing = genotypes_subset[release]['all'].count_missing(axis=0)[:]
    num_pass_missing = genotypes_subset[release]['pass'].count_missing(axis=0)[:]
    num_called = (num_variants - num_missing)
    num_pass_called = (num_pass_variants - num_pass_missing)
    print(2)
    num_het = genotypes_subset[release]['all'].count_het(axis=0)[:]
    num_pass_het = genotypes_subset[release]['pass'].count_het(axis=0)[:]
    num_hom_alt = genotypes_subset[release]['all'].count_hom_alt(axis=0)[:]
    num_pass_hom_alt = genotypes_subset[release]['pass'].count_hom_alt(axis=0)[:]
    print(3)
    num_snp_hom_ref = genotypes_subset[release]['pass'].subset(is_snp).count_hom_ref(axis=0)[:]
    num_snp_het = genotypes_subset[release]['pass'].subset(is_snp).count_het(axis=0)[:]
    num_snp_hom_alt = genotypes_subset[release]['pass'].subset(is_snp).count_hom_alt(axis=0)[:]
    num_indel_hom_ref = genotypes_subset[release]['pass'].subset(is_snp).count_hom_ref(axis=0)[:]
    num_indel_het = genotypes_subset[release]['pass'].subset(is_snp).count_het(axis=0)[:]
    num_indel_hom_alt = genotypes_subset[release]['pass'].subset(is_snp).count_hom_alt(axis=0)[:]
    print(4)    
    num_ins_hom_ref = genotypes_subset[release]['pass'].subset(is_ins).count_hom_ref(axis=0)[:]
    num_ins_het = genotypes_subset[release]['pass'].subset(is_ins).count_het(axis=0)[:]
    num_ins = (num_ins_hom_ref + num_ins_het)
    num_del_hom_ref = genotypes_subset[release]['pass'].subset(is_del).count_hom_ref(axis=0)[:]
    num_del_het = genotypes_subset[release]['pass'].subset(is_del).count_het(axis=0)[:]
    num_del = (num_del_hom_ref + num_del_het)
    
    print(5)
    pc_pass = num_pass_called / num_called
    pc_missing = num_missing / num_variants
    pc_pass_missing = num_pass_missing / num_pass_variants
    pc_het = num_het / num_called
    pc_pass_het = num_pass_het / num_pass_called
    pc_hom_alt = num_hom_alt / num_called
    pc_pass_hom_alt = num_pass_hom_alt / num_pass_called
    pc_snp = (num_snp_het + num_snp_homalt) / (num_snp_het + num_snp_homalt + num_indel_het + num_indel_homalt)
    pc_ins = (num_ins / (num_ins + num_del))

    print(6)
    df_sample_summary[release] = pd.DataFrame(
            {
                'Sample': pd.Series(samples),
                'Variants called': pd.Series(num_called),
                'Variants missing': pd.Series(num_called),
                'Proportion missing': pd.Series(pc_missing),
                'Proportion pass missing': pd.Series(pc_pass_missing),
                'Proportion heterozygous': pd.Series(pc_het),
                'Proportion pass heterozygous': pd.Series(pc_pass_het),
                'Proportion homozygous alternative': pd.Series(pc_hom_alt),
                'Proportion pass homozygous alternative': pd.Series(pc_pass_hom_alt),
                'Proportion variants SNPs': pd.Series(pc_snp),
                'Proportion indels insertions': pd.Series(pc_ins),
            }
        )

Pv30
0
0
0
0
0
0
1
2
3


ValueError: condition must be a 1-d array

In [26]:
is_ins

array([[False, False, False, False, False, False],
       [False, False, False, False, False, False],
       [False, False, False, False, False, False],
       ..., 
       [False, False, False, False, False, False],
       [False, False, False, False, False, False],
       [ True, False, False, False, False, False]], dtype=bool)

In [20]:
num_snp_hom_ref = genotypes_subset['Pv30']['pass'][is_snp, :, :].count_hom_ref(axis=0)[:]

TypeError: expected integer or slice, found: array([ True,  True,  True, ...,  True,  True, False], dtype=bool)

In [24]:
genotypes_subset['Pv30']['pass'].subset(is_snp)

<GenotypeChunkedArray shape=(963791, 1001, 2) dtype=int8 chunks=(471, 1001, 2)
   nbytes=1.8G cbytes=279.5M cratio=6.6
   compression=blosc compression_opts={'cname': 'lz4', 'shuffle': 1, 'clevel': 5}
   values=zarr.core.Array>

In [16]:
pd.value_counts(is_snp)

True     963791
False    218656
dtype: int64

In [19]:
len(is_snp)

1182447

In [7]:
is_snp = (hdf[release]['variants']['VARIANT_TYPE'][:][pass_variants] == b'SNP')

NameError: name 'pass_variants' is not defined

In [11]:
2+2

4

In [13]:
df_sample_summary['Pv30']

KeyError: 'Pv30'

In [41]:
df_sample_summary['Pf60']

,Proportion heterozygous,Proportion missing,Sample
0,0.018750,0.075449,b'FP0008-C'
1,0.009661,0.035728,b'FP0009-C'
2,0.011560,0.044328,b'FP0015-C'
3,0.005361,0.173110,b'FP0016-C'
4,0.006339,0.040628,b'FP0017-C'
5,0.008670,0.037962,b'FP0018-C'
6,0.006269,0.065103,b'FP0020-C'
7,0.006771,0.030394,b'FP0021-C'
8,0.006478,0.039455,b'FP0022-C'
9,0.007274,0.070017,b'FP0023-C'
